In [ ]:
!pip install --upgrade pip
!pip install langchain langchain-openai langchain-community langchainhub chromadb


In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma


In [ ]:
#Загрузим файлы на сервер colab
from google.colab import files
upload = files.upload()
#C:\Users\lesni\Documents\_gitprojects\llm\кейс1\main_info.md

In [ ]:
#получим список загруженных файлов
!ls

In [ ]:
!rm *.md

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import DirectoryLoader
path = ''
text_loader_kwargs={'autodetect_encoding': True}
loader = DirectoryLoader(path, glob="**/*info.md", loader_cls=TextLoader,
                         loader_kwargs=text_loader_kwargs)
docs = loader.load()
len(docs)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", "(?<=\. )"]
)

splitted_texts = splitter.split_documents(docs)
len(splitted_texts)


In [ ]:
splitted_texts[1] #увеличить размер чанка

In [ ]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")


from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

persist_directory = ''

embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(
    documents=splitted_texts,
    embedding=embedding,
    persist_directory=persist_directory
)

print(vectordb._collection.count())


In [ ]:
vectordb.delete_collection()


In [ ]:
#загрузим бд с локалки
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)
print(vectordb._collection.count())

In [ ]:
def printAnswer(answers):
    for answer in answers:
        print(answer.page_content)
        print(answer.metadata)#.source
        print('-----')

question = "Где вы находитесь?"
question = "как с вами связаться?"
answers = vectordb.similarity_search(question,k=3)

printAnswer(answers)

In [ ]:
answers = vectordb.max_marginal_relevance_search(question,k=2, fetch_k=5)
printAnswer(answers)

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain import hub

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "[REMOVED]proj-1WIRVpWSVuxZBkqHdq5OYrc0NQ9LbFfQ5Q6aHpposJJ5_gYlz0Xrpuh5p_4caMvImFYcKCgSFDT3BlbkFJIoDxqOnjzmqehbFcCzlXAIDsviQSCkDS6tcCbzOAiMCQowLg591pykL5tD3G4q9KOkK-LSRKkA"

retriever = vectordb.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\nФрагмент текста\n\n".join(doc.page_content for doc in docs)

model = ChatOpenAI(
    model="gpt-4o",
    temperature=0.7,
    )

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

rag_chain.invoke("Как с Вами связаться?")